In [50]:
import regex as re
import pandas as pd
import glob
import tabula
import pymupdf4llm
import requests
import fitz  # PyMuPDF

In [37]:
!pip install pymupdf4llm

In [38]:
base_url = "https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber="

In [39]:
inspection_ids = ["ML23129A788", "ML23067A022", "ML23128A148", "ML23124A225", "ML23082A034", "ML23067A024", "ML23115A000", "ML23048A062", "ML23122A221", "ML23128A351", "ML23089A119", "ML23108A332", "ML23124A386", "ML23088A167", "ML23102A139", "ML23114A173", "ML23059A274", "ML23114A383", "ML23124A007", "ML23123A414", "ML23054A359", "ML23122A076", "ML23089A166", "ML23080A298", "ML23102A149", "ML23023A058", "ML23060A158", "ML23101A121", "ML23074A227", "ML23030B899", "ML23122A100", "ML23116A173", "ML23025A407", "ML23072A423", "ML23068A069", "ML23121A139", "ML23023A100", "ML23110A634", "ML23116A242", "ML23068A150", "ML23124A273", "ML23089A399", "ML23089A355", "ML23102A039", "ML23122A168", "ML23044A155", "ML23075A135", "ML23095A085", "ML23123A168", "ML23116A012", "ML23062A449", "ML23110A800", "ML23095A332", "ML23089A021", "ML23073A029", "ML23129A251", "ML23111A294", "ML23118A051", "ML23116A003", "ML23130A218", "ML23038A012", "ML23118A027", "ML23118A014", "ML23125A125", "ML23026A048", "ML23122A162", "ML23124A108", "ML23125A005", "ML23118A017", "ML23104A043", "ML23130A412", "ML23088A360", "ML23053A079", "ML23124A062", "ML23103A264", "ML23086A209", "ML23069A019", "ML23128A135", "ML23122A272", "ML23053A049", "ML23053A049", "ML23121A104", "ML23130A172", "ML23128A101", "ML23102A294", "ML23116A235", "ML23121A172", "ML23075A146", "ML23123A065", "ML23066A013", "ML23122A217", "ML23052A056", "ML23129A193", "ML23103A395", "ML23044A099", "ML23102A113", "ML23033A155", "ML23121A321", "ML23074A192", "ML23121A105", "ML23110A664", "ML23032A427", "ML23075A170", "ML23114A094", "ML23131A351", "ML23123A122", "ML23132A143", "ML23132A307", "ML23047A033", "ML23114A048", "ML23121A320", "ML23129A022", "ML23107A171", "ML23025A384", "ML23110A762", "ML23068A133", "ML23123A054", "ML23032A374", "ML23102A200", "ML23111A066"]

In [40]:
inspection_id_urls_list = []

for inspection_id in inspection_ids:
    updated_id_links = base_url + inspection_id
    inspection_id_urls_list.append(updated_id_links)

inspection_id_urls_list

['https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23129A788',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23067A022',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23128A148',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23124A225',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23082A034',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23067A024',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23115A000',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23048A062',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23122A221',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23128A351',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML23089A119',
 'https://adamswebsearch2.nrc.gov/webSearch2/main.jsp?AccessionNumber=ML2310

In [41]:
inspection_ids_pdfs = [
    "ML23129A788.pdf", "ML23067A022.pdf", "ML23128A148.pdf", "ML23124A225.pdf", "ML23082A034.pdf", "ML23067A024.pdf", 
    "ML23115A000.pdf", "ML23048A062.pdf", "ML23122A221.pdf", "ML23128A351.pdf", "ML23089A119.pdf", "ML23108A332.pdf", 
    "ML23124A386.pdf", "ML23088A167.pdf", "ML23102A139.pdf", "ML23114A173.pdf", "ML23059A274.pdf", "ML23114A383.pdf", 
    "ML23124A007.pdf", "ML23123A414.pdf", "ML23054A359.pdf", "ML23122A076.pdf", "ML23089A166.pdf", "ML23080A298.pdf", 
    "ML23102A149.pdf", "ML23023A058.pdf", "ML23060A158.pdf", "ML23101A121.pdf", "ML23074A227.pdf", "ML23030B899.pdf", 
    "ML23122A100.pdf", "ML23116A173.pdf", "ML23025A407.pdf", "ML23072A423.pdf", "ML23068A069.pdf", "ML23121A139.pdf", 
    "ML23023A100.pdf", "ML23110A634.pdf", "ML23116A242.pdf", "ML23068A150.pdf", "ML23124A273.pdf", "ML23089A399.pdf", 
    "ML23089A355.pdf", "ML23102A039.pdf", "ML23122A168.pdf", "ML23044A155.pdf", "ML23075A135.pdf", "ML23095A085.pdf", 
    "ML23123A168.pdf", "ML23116A012.pdf", "ML23062A449.pdf", "ML23110A800.pdf", "ML23095A332.pdf", "ML23089A021.pdf", 
    "ML23073A029.pdf", "ML23129A251.pdf", "ML23111A294.pdf", "ML23118A051.pdf", "ML23116A003.pdf", "ML23130A218.pdf", 
    "ML23038A012.pdf", "ML23118A027.pdf", "ML23118A014.pdf", "ML23125A125.pdf", "ML23026A048.pdf", "ML23122A162.pdf", 
    "ML23124A108.pdf", "ML23125A005.pdf", "ML23118A017.pdf", "ML23104A043.pdf", "ML23130A412.pdf", "ML23088A360.pdf", 
    "ML23053A079.pdf", "ML23124A062.pdf", "ML23103A264.pdf", "ML23086A209.pdf", "ML23069A019.pdf", "ML23128A135.pdf", 
    "ML23122A272.pdf", "ML23053A049.pdf", "ML23053A049.pdf", "ML23121A104.pdf", "ML23130A172.pdf", "ML23128A101.pdf", 
    "ML23102A294.pdf", "ML23116A235.pdf", "ML23121A172.pdf", "ML23075A146.pdf", "ML23123A065.pdf", "ML23066A013.pdf", 
    "ML23122A217.pdf", "ML23052A056.pdf", "ML23129A193.pdf", "ML23103A395.pdf", "ML23044A099.pdf", "ML23102A113.pdf", 
    "ML23033A155.pdf", "ML23121A321.pdf", "ML23074A192.pdf", "ML23121A105.pdf", "ML23110A664.pdf", "ML23032A427.pdf", 
    "ML23075A170.pdf", "ML23114A094.pdf", "ML23131A351.pdf", "ML23123A122.pdf", "ML23132A143.pdf", "ML23132A307.pdf", 
    "ML23047A033.pdf", "ML23114A048.pdf", "ML23121A320.pdf", "ML23129A022.pdf", "ML23107A171.pdf", "ML23025A384.pdf", 
    "ML23110A762.pdf", "ML23068A133.pdf", "ML23123A054.pdf", "ML23032A374.pdf", "ML23102A200.pdf", "ML23111A066.pdf"
]


In [42]:
def extract_text_from_pdf(pdf_path):
    doc = pymupdf4llm.open(pdf_path)
    
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    
    return text

In [43]:
def apply_regex_to_text(text, pattern):
    return re.findall(pattern, text)

In [44]:
inspectdate_pat = r"(?<=completed an inspection\s+at.*?)(\w+\s+\d{1,2},\s+\d{4})"
plantname_pat = r"SUBJECT:\s(\w+(\s\w+)*)"
violation_pat = r"(?:violation|apparent violation.*?)(?:\(\w+\))?"

In [45]:
df = pd.DataFrame(columns=['Plant Name', 'Inspection Date', 'Violation?'])

In [46]:
pdf_files = glob.glob("*.pdf")

In [48]:
pip install PyMuPDF


Note: you may need to restart the kernel to use updated packages.


In [54]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    
    return text

data = []

pdf_files = glob.glob("*.pdf")

for pdf_file in pdf_files:
    text = extract_text_from_pdf(pdf_file)
    
    plant_name = re.findall(plantname_pat, text)
    date = re.findall(inspectdate_pat, text)
    violation = re.findall(violation_pat, text)
    
    if plant_name and date and violation:
        data.append({
            'Plant Name': plant_name[0],
            'Inspection Date': date[0],
            'Violation?': violation[0]
        })
    else:
        data.append({
            'Plant Name': "n/a",
            'Inspection Date': "n/a",
            'Violation?': "n/a"
        })

df = pd.DataFrame(data)

print(df)


                                    Plant Name   Inspection Date Violation?
0                               (DONALD C,  C)  January 17, 2023  violation
1                                          n/a               n/a        n/a
2                                          n/a               n/a        n/a
3          (POINT BEACH NUCLEAR PLANT,  PLANT)    April 10, 2023  violation
4                                          n/a               n/a        n/a
..                                         ...               ...        ...
102                                        n/a               n/a        n/a
103  (DRESDEN NUCLEAR POWER STATION,  STATION)    April 12, 2023  violation
104     (SHEARON HARRIS NUCLEAR PLANT,  PLANT)       May 8, 2023  violation
105                                        n/a               n/a        n/a
106                                        n/a               n/a        n/a

[107 rows x 3 columns]


In [55]:
df.to_csv('nucplantviolations.csv', index=False)